In [2]:
import pypico

In [4]:
pico = pypico.load_pico("/Users/yy/git/my_PHYS321_final/data/pico4_tailmonty_v35_py3.dat") 
# Note i didn't upload the data file to github as it's too large
# change to your own path to see data

In [5]:
pico.inputs()

{'Alens',
 'helium_fraction',
 'massive_neutrinos',
 'ombh2',
 'omch2',
 'omnuh2',
 'pivot_scalar',
 're_optical_depth',
 'scalar_amp(1)',
 'scalar_nrun(1)',
 'scalar_spectral_index(1)',
 'theta'}

In [6]:
pico.outputs()

['cl_TT',
 'cl_TE',
 'cl_EE',
 'cl_BB',
 'k',
 'scalar_TE',
 'scalar_EE',
 'scalar_TT',
 'cl_pT',
 'lensed_EE',
 'lensed_BB',
 'pk',
 'lensed_TT',
 'lensed_TE',
 'cl_pp']